<div style="background-color:#e8f5e9; border-left: 6px solid #4caf50; padding: 15px; border-radius: 5px;">
    <h1 style="color:#2e7d32;">FIT5149 2024 S2 Assignment 1</h1>

    Python Environment: 3.10.0

    Libraries required can be found in the requirements.txt file.
</div>


<div style="background-color:#e3f2fd; border-left: 6px solid #2196f3; padding: 15px; border-radius: 5px; width: fit-content;">
    <h2 style="color:#1976d2; margin-bottom: 5px;">Student Information</h2>
    <p style="color:#333; font-size: 16px; margin: 0;"><strong>Student ID:</strong> 26921677</p>
    <p style="color:#333; font-size: 16px; margin: 0;"><strong>Name:</strong> Zachary Plischka</p>
    <p style="color:#333; font-size: 16px; margin: 0;"><strong>Group:</strong> 70</p>
</div>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
# turn warnings off
import warnings
warnings.filterwarnings("ignore")

## Inital Data Exploration and Analyisis:

In [2]:
#Load the data
pd.set_option('display.max_columns', None)
kag = pd.read_csv('data/A1_stock_volatility_kaggle.csv')
df = pd.read_csv('data/A1_stock_volatility_labeled.csv')
sub = pd.read_csv('data/A1_stock_volatility_submission.csv')
df.columns = df.columns.str.lower()

df['target'] = df.groupby('stock')['volatility'].shift(-1)

In [ ]:
#Using pandas profiling to get an overview of the data and the distribution of the features
# profile = ProfileReport(df, title="Pandas Profiling Report")
# profile.to_notebook_iframe()

In [4]:
# Determine the columns that are snapshot and cumulative, also monthly features and columns that are right skewed
snapshot_quarterly = ['revenue', 'net income', 'gross profit', 'eps', 'total assets', 'total liabilities', 'total equity', 'cash and cash equivalents']
cumulative_quarterly = ['operating cash flow', 'investing cash flow', 'financing cash flow']
monthly_features = ['volatility', 'open', 'close', 'high', 'low', 'volume', 'amount', 'avg_price', 'return']
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
right_skewed = ['open', 'close', 'high', 'gross profit', 'revenue', 'low', 'volume', 'amount', 'avg_price', 'net income', "total assets", "total liabilities", "total equity", "cash and cash equivalents", 'operating cash flow']
negative_cols = df[num_cols].loc[:, (df[num_cols] < 0).any()].columns.tolist()
ltc = [x for x in right_skewed if x not in negative_cols] # log transform columns
original_cols = df.columns

def preprocessing(data, remove_outliers=False):
    """
    Preprocesses the given data by performing the following steps:
    1. Creates a copy of the data.
    2. If remove_outliers is True, deals with outliers by performing log transformation on specified columns, replacing extreme values with medians, and dropping rows with extreme values.
    3. Creates lagged features for volatility and return.
    4. Fills missing values in lagged features with the mean volatility and return for each stock.
    5. Creates high_low_ratio feature.
    6. Calculates rolling mean for volatility.
    7. Fills missing values in volume_pct_change with the mean value for each stock.
    8. Calculates P/E ratio.
    9. Calculates log of volume.
    10. Calculates volatility change.
    11. Replaces infinite values with NaN.
    12. Creates test_df by selecting the last row for each stock.
    13. Drops rows with missing values from df.
    14. Returns the preprocessed df and test_df.
    Parameters:
    - data: DataFrame, the input data to be preprocessed.
    - remove_outliers: bool, indicates whether to remove outliers or not. Default is False.
    Returns:
    - df: DataFrame, the preprocessed data.
    - test_df: DataFrame, the test data consisting of the last row for each stock.
    """

    df = data.copy()
    
    # if remove_outliers is True, this will deal with outliers
    if remove_outliers:
        def log_transform_cols(data, cols):
            for col in cols:
                data[col] = np.log(data[col])
            return data
        
        df = log_transform_cols(df, ltc)
        df['volatility'] = np.where(df['volatility'] > 100, df['volatility'].median(), df['volatility'])
        df['return'] = np.where(df['return'] > 150, df['return'].median(), df['return'])
        df = df.drop(df[(df['volatility'] > 100) | (df['target'] > 100)].index)
    
    
    # Create lagged features
    df['vola_lag2'] = df.groupby('stock')['volatility'].shift(1)
    df['vola_lag3'] = df.groupby('stock')['volatility'].shift(2)

    # Fill missing values with the mean volatility for each stock
    df['vola_lag2'] = df.groupby('stock')['vola_lag2'].transform(lambda x: x.fillna(x.mean()))
    df['vola_lag3'] = df.groupby('stock')['vola_lag3'].transform(lambda x: x.fillna(x.mean()))

    # Create lagged return features
    df['return_lag2'] = df.groupby('stock')['return'].shift(1)
    df['return_lag3'] = df.groupby('stock')['return'].shift(2)

    # fill na with mean
    df['return_lag2'] = df.groupby('stock')['return_lag2'].transform(lambda x: x.fillna(x.mean()))
    df['return_lag3'] = df.groupby('stock')['return_lag3'].transform(lambda x: x.fillna(x.mean()))

    # Create high_low_ratio feature
    df['high_low_ratio'] = df['high'] / df['low']

    # ROLLING FEATURES
    df['vola_rolling_3'] = df.groupby('stock')['volatility'].rolling(3).mean().reset_index(0, drop=True)
    # fill na with mean
    df['vola_rolling_3'] = df.groupby('stock')['vola_rolling_3'].transform(lambda x: x.fillna(x.mean()))
    
    df['volume_pct_change'] = df.groupby('stock')['volume'].pct_change()
    df['volume_pct_change'] = df.groupby('stock')['volume_pct_change'].transform(lambda x: x.fillna(x.mean()))

    # P/E RATIO
    df['pe_ratio'] = df['avg_price'] / df['eps']

    df['log_volume'] = np.log(df['volume'])

    df['volatility_change'] = (df['volatility'] - df['vola_lag2']).abs() + (df['vola_lag2'] - df['vola_lag3']).abs()

    df = df.replace([np.inf, -np.inf], np.nan)

    
    #------------------------------------------------ Final Drop and creation of Test_df

    
    test_df = df.groupby('stock').tail(1)
    df = df.copy()
    df.dropna(inplace=True)
    
    

    return df, test_df

<div style="background-color:#e6f7ff; color:#000000; padding:15px; border-radius:5px; border: 1px solid #cce7ff;">
  
## Initial EDA and Beginning of Analysis Task

Initial analysis of the dataset shows that there are some significant outliers in the dataset that need to be dealt with accordingly. Additionally, the dataset contains variables that are highly correlated with each other. This can be seen in both the correlation tab and the 'alerts' tab in the HTML output above.

Columns that are highly skewed include:
- open
- close
- high
- low
- avg_price
- volatility
- target

The missing values detected in the dataset are in the 'target' column. This is because the target column is created by using next month's volatility. Since the last month of the dataset does not have a next month, the target column is missing a value for the last month. Therefore, this is the value that we ultimately want to predict. Since the data in the same row is not missing and will ideally contain historical information, we can use this data to predict the target value.

</div>


#  Comparing Outliers dealt with vs NOT dealt with

In [5]:
dealt_with, _ = preprocessing(df, remove_outliers=True)
not_dealt_with, _ = preprocessing(df, remove_outliers=False)

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import math

def truncate_label(label, max_length=40):
    return label[:max_length] + '...' if len(label) > max_length else label

def plot_features_grid(df, target, cols_per_row=4):
    """Plot a grid of scatter plots showing the relationship between features and a target variable.
    Parameters:
    - df (pandas.DataFrame): The DataFrame containing the data.
    - target (str): The name of the target variable.
    - cols_per_row (int, optional): The number of columns per row in the grid. Default is 4.
    Returns:
    None
    """ 
    features = [col for col in df.columns if col != target]
    n_features = len(features)
    n_rows = math.ceil(n_features / cols_per_row)
    
    # Increase figure size and adjust subplot parameters
    fig, axes = plt.subplots(n_rows, cols_per_row, figsize=(5*cols_per_row, 4*n_rows))
    fig.suptitle(f'Features vs {target}', fontsize=16)
    plt.subplots_adjust(hspace=0.4, wspace=0.4)
    
    axes = axes.flatten() if isinstance(axes, np.ndarray) else [axes]
    
    for i, feature in enumerate(features):
        ax = axes[i]
        
        sns.scatterplot(x=feature, y=target, data=df, ax=ax)
        ax.set_title(truncate_label(feature), fontsize=10)
        ax.set_xlabel(truncate_label(feature), fontsize=8)
        ax.set_ylabel(target, fontsize=8)
        
        # Limit the number of x-ticks and truncate their labels
        ax.xaxis.set_major_locator(plt.MaxNLocator(5))
        ax.set_xticklabels([truncate_label(label.get_text(), 10) for label in ax.get_xticklabels()])
        
        ax.tick_params(axis='both', which='major', labelsize=6)
    
    # Remove any unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

# Use the function
plot_features_grid(not_dealt_with, 'target')

<div style="background-color:#f0f8ff; color:#000000; padding:15px; border-radius:5px; border: 1px solid #d3e0ea;">
  
## Analysis of Outliers

As we can see in the plots above, there is a clear outlier that has a target value of ~140, which is noticeable in every plot. Since the target variable was derived by shifting the volatility variable, we can also observe a few outliers in the volatility-related plots. Additionally, there are two potential outliers in the 'return' derived plots such as 'return_lag2', 'return_lag3', and 'pe_ratio'.

Regarding the price-related variables such as 'open', 'close', 'high', 'low', and 'avg_price', there is a distinct cluster of potential outliers with prices significantly higher than the rest of the dataset. Upon further exploration, it appears that these data points are related to Walmart ("WMT") stock. These outliers can be addressed by applying transformations to the features. A potential transformation is the log transformation, which could compress larger values and reduce the right skew of the data.

</div>


In [ ]:
not_dealt_with[not_dealt_with['open']> 300000]

In [ ]:
not_dealt_with[not_dealt_with['volatility_change']> 100]

## Analysis of dataset with outliers dealt with:

In [9]:
plot_features_grid(dealt_with, 'target')

<div style="background-color:#f1f8e9; color:#000000; padding:15px; border-radius:5px; border-left: 6px solid #81c784; padding: 15px;">

## Dealing with Outliers vs Not Dealing with Outliers:
As we can see from the example below, the baseline model appears to perform better on the dataset where the outliers are dealt with, as opposed to leaving them in the dataset.

</div>


In [ ]:
# Train a linear regression model and compare the performance with and without removing outliers

from sklearn.linear_model import LinearRegression

not_removed, not_removed_test = preprocessing(df, remove_outliers=False)
removed, removed_test = preprocessing(df, remove_outliers=True)

X_not_removed = not_removed.drop(columns=['target', 'stock', 'date'])
y_not_removed = not_removed['target']

X_removed = removed.drop(columns=['target', 'stock', 'date'])
y_removed = removed['target']

# train test split
from sklearn.model_selection import train_test_split

X_train_not_removed, X_val_not_removed, y_train_not_removed, y_val_not_removed = train_test_split(X_not_removed, y_not_removed, test_size=0.2, random_state=42)
X_train_removed, X_val_removed, y_train_removed, y_val_removed = train_test_split(X_removed, y_removed, test_size=0.2, random_state=42)

# train linear regression model
lr_not_removed = LinearRegression()
lr_removed = LinearRegression()

lr_not_removed.fit(X_train_not_removed, y_train_not_removed)
lr_removed.fit(X_train_removed, y_train_removed)

# make predictions
y_pred_not_removed = lr_not_removed.predict(X_val_not_removed)
y_pred_removed = lr_removed.predict(X_val_removed)

# evaluate model
from sklearn.metrics import mean_squared_error

mse_not_removed = mean_squared_error(y_val_not_removed, y_pred_not_removed)
mse_removed = mean_squared_error(y_val_removed, y_pred_removed)

print(f'MSE not removed: {mse_not_removed}')
print(f'MSE removed: {mse_removed}')

# r2 score
from sklearn.metrics import r2_score

r2_not_removed = r2_score(y_val_not_removed, y_pred_not_removed)
r2_removed = r2_score(y_val_removed, y_pred_removed)

print(f'R2 not removed: {r2_not_removed}')
print(f'R2 removed: {r2_removed}')

%matplotlib inline
import matplotlib.pyplot as plt

# plot predictions vs actual
def plot_predictions_vs_actual(y_true, y_pred, title):
    plt.figure(figsize=(10, 6))
    plt.scatter(y_true, y_pred, alpha=0.6)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=2)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title(title)
    plt.show()
    
plot_predictions_vs_actual(y_val_not_removed, y_pred_not_removed, 'Predictions vs Actual (Not Removed)')
plot_predictions_vs_actual(y_val_removed, y_pred_removed, 'Predictions vs Actual (Removed)')

<div style="background-color:#e8f4f8; color:#000000; padding:15px; border-radius:5px; border: 1px solid #b0d4de;">
  
Dealing with the outliers in an appropriate manner has reduced the variance of our predictions, as shown by the decrease in the $R^2$ score and subsequently the decrease in the MSE.

</div>


<div style="background-color:#f9f9f9; color:#000000; padding:15px; border-radius:5px; border: 1px solid #dddddd;">

## Begin Polynomial and Interaction Feature Engineering:

The formation of polynomial features allows us to capture non-linear relationships between the input variables and the target variable. By creating new features here, the model can learn more complex patterns in the data.

</div>


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# show max columns
pd.set_option('display.max_columns', None)
kag = pd.read_csv('data/A1_stock_volatility_kaggle.csv')
df = pd.read_csv('data/A1_stock_volatility_labeled.csv')
sub = pd.read_csv('data/A1_stock_volatility_submission.csv')
df.columns = df.columns.str.lower()
df

df['target'] = df.groupby('stock')['volatility'].shift(-1)
df.columns = df.columns.str.lower()

## Preprocessing + Feature Engineering


In [12]:
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

def create_polynomial_features(data, degree=2, exclude_columns=None):
    """
    Generate polynomial features for the given data.
    Parameters:
    - data (pandas.DataFrame): The input data.
    - degree (int): The degree of the polynomial features. Default is 2.
    - exclude_columns (list): List of column names to exclude from polynomial feature generation. Default is None.
    Returns:
    - result (pandas.DataFrame): The data with polynomial features generated.
    """
    
    if exclude_columns is None:
        exclude_columns = []
    
    # Separate excluded columns
    excluded_data = data[exclude_columns]
    features = data.drop(columns=exclude_columns)

    poly = PolynomialFeatures(degree=degree, include_bias=False)
    poly_features = poly.fit_transform(features)
    
    # Check if get_feature_names_out method exists (newer scikit-learn versions)
    if hasattr(poly, 'get_feature_names_out'):
        poly_feature_names = poly.get_feature_names_out(features.columns)
    # Fall back to get_feature_names for older versions
    elif hasattr(poly, 'get_feature_names'):
        poly_feature_names = poly.get_feature_names(features.columns)
    else:
        # If neither method is available, create generic feature names
        n_features = poly_features.shape[1]
        poly_feature_names = [f'feature_{i}' for i in range(n_features)]
    
    df_poly = pd.DataFrame(poly_features, columns=poly_feature_names, index=features.index)
    
    # Combine polynomial features with excluded columns
    result = pd.concat([df_poly, excluded_data], axis=1)
    
    return result

### Applying the transformations to the Data:

In [ ]:
# Preprocess the data and create polynomial features
processed_df, test_df = preprocessing(df, remove_outliers=True)

# For df: exclude 'stock', 'date', and 'target', then add back 'stock' and 'target'
df_poly = create_polynomial_features(processed_df, degree=2, exclude_columns=['stock', 'date', 'target'])
df_poly = df_poly.drop(columns='date')  # Remove 'date' if it's not needed

# For test_df: exclude 'stock' and 'date', then add back 'stock'
test_df = create_polynomial_features(test_df, degree=2, exclude_columns=['stock', 'date', 'target'])
test_df = test_df.drop(columns='date')  # Remove 'date' if it's not needed

print(df_poly.columns)
print(test_df.columns)

## Plotting and analysing the new features that have been created using scatterplots.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import math

def plot_features(df, target, cols_per_row=4):
    """
    Plot the feature analysis against the target variable.
    Parameters:
    - df (pandas.DataFrame): The input DataFrame containing the features and target variable.
    - target (str): The name of the target variable.
    - cols_per_row (int, optional): The number of columns per row in the subplots. Default is 4.
    Returns:
    None
    """
    
    features = [col for col in df.columns if col != target]
    n_features = len(features)
    n_rows = math.ceil(n_features / cols_per_row)
    
    # Reduce the figure size
    fig, axes = plt.subplots(n_rows, cols_per_row, figsize=(16, 4 * n_rows))
    fig.suptitle(f'Feature Analysis vs {target}', fontsize=16)
    
    for i, col in enumerate(features):
        row = i // cols_per_row
        col_pos = i % cols_per_row
        ax = axes[row, col_pos] if n_rows > 1 else axes[col_pos]
        
        if pd.api.types.is_numeric_dtype(df[col]):
            # For numerical columns
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = (df[col] < lower_bound) | (df[col] > upper_bound)
            
            sns.scatterplot(data=df[~outliers], x=col, y=target, color='blue', alpha=0.6, ax=ax, s=10)
            sns.scatterplot(data=df[outliers], x=col, y=target, color='red', alpha=0.6, ax=ax, s=10)
        else:
            # For categorical columns
            sns.boxplot(data=df, x=col, y=target, ax=ax)
        
        ax.set_title(f'{col} vs {target}', fontsize=10)
        ax.tick_params(axis='both', which='major', labelsize=8)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    
    # Remove any unused subplots
    for j in range(i+1, n_rows * cols_per_row):
        row = j // cols_per_row
        col_pos = j % cols_per_row
        fig.delaxes(axes[row, col_pos] if n_rows > 1 else axes[col_pos])
    
    plt.tight_layout()
    plt.show()


plot_features(df_poly, 'target')

Briefly analysing the visuals above, we can see that some new features have a strong linear correlation with the target, this can potentially improve the performance of the models.

<div style="background-color:#f4f6fa; color:#000000; padding:15px; border-radius:5px; border: 1px solid #c9d3e0;">

## Feature Selection:

Instead of using MSE or $R^2$ to evaluate feature selection, we use BIC. This is because MSE decreases monotonically as feature size increases, and $R^2$ increases monotonically as feature size increases.

Below, we conduct forward feature selection on the transformed dataset using a simple linear regression model.

**NOTE**: This cell can take upwards of 20 minutes to run. You can skip this cell and use the best features that were extracted below.

</div>


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def calculate_bic(y_true, y_pred, n_features):
    """
    Calculate the Bayesian Information Criterion (BIC) for a regression model.
    Parameters:
    - y_true (array-like): The true values of the target variable.
    - y_pred (array-like): The predicted values of the target variable.
    - n_features (int): The number of features used in the regression model.
    Returns:
    - bic (float): The calculated BIC value.
    Notes:
    - The BIC is a criterion for model selection among a finite set of models.
    - It penalizes models with more features to prevent overfitting.
    - The BIC is calculated as n_samples * log(mse) + n_features * log(n_samples),
      where mse is the mean squared error between y_true and y_pred.
    - If the mean squared error is less than or equal to 0, the BIC is set to infinity.
    """
    
    n_samples = len(y_true)
    mse = mean_squared_error(y_true, y_pred)
    if mse <= 0:
        return np.inf
    bic = n_samples * np.log(mse) + n_features * np.log(n_samples)
    return bic

# Split the data
X = df_poly.drop(columns=['target', 'stock']).reset_index(drop=True)
y = df_poly['target']

# Create a pipeline with a standard scaler and a linear regression model
pipeline = make_pipeline(StandardScaler(), LinearRegression())

# Initialize lists to store results
n_features_list = []
bic_scores = []
selected_features_list = []

# Initialize set of selected features
selected_features = set()
remaining_features = set(X.columns)

# Perform forward feature selection
for n_features in range(1, 101):
    best_bic = np.inf
    best_feature = None
    
    # Try adding each remaining feature
    for feature in remaining_features:
        current_features = list(selected_features) + [feature]
        X_selected = X[current_features]
        
        # Evaluate the model with the current set of features
        pipeline.fit(X_selected, y)
        y_pred = pipeline.predict(X_selected)
        bic = calculate_bic(y, y_pred, n_features)
        
        # Update best feature if this one gives a lower BIC
        if bic < best_bic:
            best_bic = bic
            best_feature = feature
    
    # Add the best feature to the selected set
    selected_features.add(best_feature)
    remaining_features.remove(best_feature)
    
    # Store the results
    n_features_list.append(n_features)
    bic_scores.append(best_bic)
    selected_features_list.append(list(selected_features))
    
    print(f"Number of features: {n_features}")
    print(f"Selected features: {list(selected_features)}")
    print(f"BIC Score: {best_bic:.4f}")
    print("--------------------")

# Create a DataFrame with the results
results = pd.DataFrame({
    'n_features': n_features_list,
    'bic': bic_scores,
    'selected_features': selected_features_list
})

# Find the best number of features
best_n_features = results.loc[results['bic'].idxmin(), 'n_features']
best_features = results.loc[results['bic'].idxmin(), 'selected_features']

print(f"\nBest number of features: {best_n_features}")
print(f"Best features: {best_features}")
print(f"Best BIC: {results['bic'].min():.4f}")

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(results['n_features'], results['bic'], marker='o')
plt.xlabel('Number of Features')
plt.ylabel('BIC Score')
plt.title('Forward Feature Selection: BIC vs Number of Features')
plt.show()

<div style="background-color:#f5f7fb; color:#000000; padding:15px; border-radius:5px; border: 1px solid #d0d7e5;">

## Analysis of Feature Selection:

As shown in the above plot, there is a dramatic decrease in the BIC score from 0 to ~40 features, where the scoring begins to plateau. The best number of features that resulted from the forward feature selection was 54, with a BIC of 9.91.

</div>


In [16]:
best_features = ['return_lag2 high_low_ratio',
 'revenue vola_lag3',
 'return_lag3 vola_rolling_3',
 'volatility volume_pct_change',
 'return volatility',
 'volume vola_lag2',
 'cash and cash equivalents vola_lag2',
 'eps return_lag3',
 'return_lag2 volatility_change',
 'log_volume volatility_change',
 'return_lag3 high_low_ratio',
 'return vola_lag3',
 'return cash and cash equivalents',
 'low return',
 'vola_rolling_3',
 'vola_lag3 log_volume',
 'avg_price volatility_change',
 'amount revenue',
 'vola_lag2 vola_lag3',
 'total liabilities vola_rolling_3',
 'close volatility_change',
 'return investing cash flow',
 'vola_lag2 return_lag3',
 'vola_rolling_3 volatility_change',
 'return return_lag2',
 'return gross profit',
 'volatility vola_lag2',
 'volatility volatility_change',
 'volume vola_lag3',
 'volatility investing cash flow',
 'return return_lag3',
 'low volatility_change',
 'cash and cash equivalents vola_rolling_3',
 'return_lag2^2',
 'return_lag3^2',
 'return total liabilities',
 'volatility vola_rolling_3',
 'high volatility_change',
 'high_low_ratio vola_rolling_3',
 'volatility return_lag2',
 'investing cash flow^2',
 'avg_price return',
 'close return',
 'revenue vola_lag2',
 'total assets volatility_change',
 'vola_rolling_3^2',
 'total assets vola_rolling_3',
 'return^2',
 'open volatility_change']


<div style="background-color:#f7fafc; color:#000000; padding:15px; border-radius:5px; border: 1px solid #d6dee7;">

## Model Training

Using the best features obtained from the feature selection, we set up a grid search, random search, and Bayesian search to tune the hyperparameters of the different models we wish to train.

**Note**: Ideally, we would conduct feature selection on each model after training them. However, due to limited computational resources, this approach seemed optimal.

</div>


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from scipy.stats import uniform, randint
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import mutual_info_regression

# Split data
X = df_poly[best_features]
y = df_poly['target']

pipelines = {
    'ridge': make_pipeline(StandardScaler(), Ridge(random_state=42)),
    'xgb': XGBRegressor(random_state=42),  # Remove StandardScaler for XGBoost
    'rf': make_pipeline(StandardScaler(), RandomForestRegressor(random_state=42)),
    'svr': make_pipeline(StandardScaler(), SVR()),
    'knn': make_pipeline(StandardScaler(), KNeighborsRegressor())
}

# Create alphas
alphas = np.logspace(-4, 4, 9)

grid = {
    'ridge': {
        'ridge__alpha': alphas,
    },
    'rf': {
        'randomforestregressor__n_estimators': [100, 200],
        'randomforestregressor__max_depth': [3, 5, 7],
    },
    'svr': {
        'svr__C': [1, 10],
        'svr__kernel': ['rbf']
    },
    'knn': {
        'kneighborsregressor__n_neighbors': [3, 5, 7],
    }
}

# XGBoost optimized search parameters
xgb_param_distributions = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_weight': randint(1, 10),
    'gamma': uniform(0, 0.5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
}

xgb_search_spaces = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 10),
    'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
    'subsample': Real(0.6, 1.0),
    'colsample_bytree': Real(0.6, 1.0),
    'min_child_weight': Integer(1, 10),
    'gamma': Real(0, 0.5),
    'reg_alpha': Real(0, 1),
    'reg_lambda': Real(0, 1),
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Starting model fitting and evaluation...")
fit_models = {}

for algo in pipelines.keys():
    print(f"Fitting {algo}...")
    
    if algo == 'xgb':
        # Step 1: Initial exploration with RandomizedSearchCV
        random_search = RandomizedSearchCV(
            estimator=pipelines[algo],
            param_distributions=xgb_param_distributions,
            n_iter=50,  
            cv=3,
            verbose=2,
            random_state=42,
            n_jobs=-1
        )
        random_search.fit(X_train, y_train)
        print("Best parameters found by RandomizedSearchCV:")
        print(random_search.best_params_)
        
        # Step 2: Fine-tuning with BayesianOptimization
        bayes_search = BayesSearchCV(
            estimator=XGBRegressor(random_state=42),
            search_spaces=xgb_search_spaces,
            n_iter=25, 
            cv=3,
            verbose=2,
            random_state=42,
            n_jobs=-1
        )
        bayes_search.fit(X_train, y_train)
        model = bayes_search
    else:
        model = GridSearchCV(pipelines[algo], grid[algo], cv=3, n_jobs=-1, verbose=1)
        model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mse = np.sqrt(mean_squared_error(y_test, y_pred))
    fit_models[algo] = model
    print(f'{algo} RMSE: {mse}')
    print(f'Best parameters for {algo}: {model.best_params_}')
    print("--------------------")





In [ ]:
# print rmse for each model
for algo, model in fit_models.items():
    y_pred = model.predict(X_test)
    mse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f'{algo} RMSE: {mse}')

<div style="background-color:#f9fbfd; color:#000000; padding:15px; border-radius:5px; border: 1px solid #cfd7e2;">

## Selecting the Best Model:

After tuning the hyperparameters on the models and employing 3-fold cross-validation throughout the process, the best performing model was the Ridge Regressor with an alpha value of 10.

</div>


<div style="background-color:#f4f8fc; color:#000000; padding:15px; border-radius:5px; border: 1px solid #d0d8e5;">

## Feature Importance:

In this section, I will perform feature importance analysis to identify the key features that the models are utilizing.

</div>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# plot feature importances
for algo, model in fit_models.items():
    if algo not in ['xgb', 'rf', 'ridge']:
        continue  

    plt.figure(figsize=(10, 6))
    plt.title(f"Feature Importances ({algo.upper()})")
    
    if algo in ['xgb', 'rf']:
        if algo == 'xgb':
            importances = model.best_estimator_.feature_importances_
        elif algo == 'rf':
            importances = model.best_estimator_.named_steps['randomforestregressor'].feature_importances_
    elif algo == 'ridge':
        importances = np.abs(model.best_estimator_.named_steps['ridge'].coef_)
    
    indices = np.argsort(importances)[::-1]
    
    plt.bar(range(len(importances)), importances[indices])
    plt.xticks(range(len(importances)), [best_features[i] for i in indices], rotation=90)
    plt.tight_layout()
    plt.show()

In [ ]:
# plot r2 score for each model
from sklearn.metrics import r2_score
r2_scores = {}
for algo, model in fit_models.items():
    y_pred = model.predict(X_test)
    r2_scores[algo] = r2_score(y_test, y_pred)

plt.figure(figsize=(10, 6))
plt.bar(r2_scores.keys(), r2_scores.values())
plt.xlabel('Algorithm')
plt.ylabel('R2 Score')
plt.title('R2 Score for Each Model')
plt.show()




<div style="background-color:#f8fbff; color:#000000; padding:15px; border-radius:5px; border: 1px solid #cfd9e6;">

### Feature Importance Analysis:

Based on the analysis above, volatility-related features are observed to be the most important features used by the models. Specifically, the rolling averages of the past 3 months stood out, as well as its squared counterpart. This can be demonstrated by the larger coefficients observed in the Ridge Regression model and the feature importance scores extracted from the XGB and RandomForest models.

</div>


<div style="background-color:#f7faff; color:#000000; padding:15px; border-radius:5px; border: 1px solid #d3dcea;">

## Residual Analysis

**THINGS TO LOOK FOR**

- **Patterns**: Look for any distinct patterns in the residuals.
  
- **Spread**: The spread of residuals should be fairly consistent across the range of each feature. Inconsistencies might indicate heteroscedasticity.

- **Outliers**: Identify any points far from the main cluster, especially if they appear in multiple plots.

- **Trend Lines**: Ideally, each trend line should be close to horizontal.

</div>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

# Get the Ridge model and make predictions
ridge_model = fit_models['ridge']
y_pred = ridge_model.predict(X_test)
residuals = y_test - y_pred

# Function to create a single residual plot
def plot_residuals(ax, x, residuals, title):
    """
    Plot the residuals against a given variable.
    Parameters:
    - ax (matplotlib.axes.Axes): The axes to plot on.
    - x (array-like): The variable to plot against the residuals.
    - residuals (array-like): The residuals to be plotted.
    - title (str): The title of the plot.
    Returns:
    None
    """
    
    ax.scatter(x, residuals, alpha=0.5, s=10)
    ax.axhline(y=0, color='r', linestyle='-', linewidth=1)
    ax.set_xlabel(title, fontsize=8)
    ax.set_ylabel('Residuals', fontsize=8)
    ax.set_title(f'Residuals vs {title}', fontsize=10)
    ax.tick_params(axis='both', which='major', labelsize=6)
    
    # Add a trend line
    z = np.polyfit(x, residuals, 1)
    p = np.poly1d(z)
    ax.plot(x, p(x), "r--", alpha=0.8, linewidth=1)

# Calculate number of rows needed (4 plots per row)
n_features = len(X_test.columns) + 2  # +2 for predicted and actual values
n_rows = math.ceil(n_features / 4)

# Create subplots
fig, axs = plt.subplots(n_rows, 4, figsize=(20, 5*n_rows))
fig.suptitle('Residual Plots for Ridge Model', fontsize=16)

# Flatten axs if it's a 2D array
axs = axs.flatten() if n_rows > 1 else axs

# Plot residuals for each feature
for i, feature in enumerate(X_test.columns):
    plot_residuals(axs[i], X_test[feature], residuals, feature)

# Plot residuals vs predicted values
plot_residuals(axs[-2], y_pred, residuals, 'Predicted Target')

# Plot residuals vs actual values
plot_residuals(axs[-1], y_test, residuals, 'Actual Target')

# Remove any unused subplots
for i in range(n_features, len(axs)):
    fig.delaxes(axs[i])

plt.tight_layout()
plt.subplots_adjust(top=0.95)  # Adjust to prevent title overlap
plt.show()

<div style="background-color:#f5f9ff; color:#000000; padding:15px; border-radius:5px; border: 1px solid #d0d8e5;">

## Residual Analysis:

Overall, the visualizations of the residuals are relatively good as most of the trend lines are very close to horizontal, and not too many outliers are present. This indicates a favorable environment for a linear regression model.

</div>


<div style="background-color:#f5f9ff; color:#000000; padding:15px; border-radius:5px; border: 1px solid #d0d8e5;">

## Make predictions and save submission file


In [ ]:
sub

In [23]:
best_model = fit_models['ridge']
pred_df = test_df[best_features]
sub['Volatility'] = best_model.predict(pred_df)
if 'Date' in sub.columns:
    sub.drop(columns=['Date'], inplace=True)


In [25]:
sub.to_csv('pred_values.csv', index=False)